In [ ]:
import pandas as pd
import numpy as np
import jieba
import keras
from keras_bert import get_base_dict, get_model, compile_model, gen_batch_inputs
from keras_bert import load_trained_model_from_checkpoint
from collections import namedtuple
from keras.utils.np_utils import *
import tensorflow as tf

In [ ]:
from keras.backend.tensorflow_backend import set_session
config =tf.compat.v1.ConfigProto(allow_soft_placement=True)
#config.gpu_options.per_process_gpu_memory_fraction = 0.3
config.log_device_placement = True
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
sess =tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(tf.compat.v1.Session(config=config))

In [ ]:
import os
from keras_bert import Tokenizer
pretrained_path  = "bert/"
EPOCH = 10
BATCH_SIZE = 16
SEQ_LEN = 120
MAX_LEN = 384
MODEL_NAME = 'bert_model.h5'
PretrainedPaths = namedtuple('PretrainedPaths', ['config', 'model', 'vocab'])
config_path = os.path.join(pretrained_path, 'bert_config.json')
model_path = os.path.join(pretrained_path, 'bert_model.ckpt')
vocab_path = os.path.join(pretrained_path, 'vocab.txt')
paths = PretrainedPaths(config_path, model_path, vocab_path)
#tokenizer = Tokenizer(paths.vocab)
from keras_bert import load_vocabulary
token_dict = load_vocabulary(vocab_path)

tokenizer = Tokenizer(token_dict)
text = '语言模型'
tokens = tokenizer.tokenize(text)
print(tokens)
indices, segments = tokenizer.encode(first=text, max_len=120)
print(indices[:10])
# [101, 6427, 6241, 3563, 1798, 102, 0, 0, 0, 0]
print(segments[:10])
# [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
from keras_bert.layers import Extract
model = load_trained_model_from_checkpoint(
config_file=paths.config,
checkpoint_file=paths.model,
seq_len = MAX_LEN,
trainable = False
)

In [ ]:
last = model.output
extract = Extract(index=-1, name='Extract')(last)
dense = keras.layers.Dense(units=768, name='Dense')(extract)
norm = keras.layers.BatchNormalization(name='Normal')(dense)
output = keras.layers.Dense(units=2, activation='softmax', name='Softmax')(norm)
model = keras.models.Model(inputs=model.inputs, outputs=output)
model.summary()

In [ ]:
from keras.optimizers import Adam
from keras import metrics
model.compile(Adam(lr=2e-6), loss='binary_crossentropy', metrics=['accuracy'])
#categorical_crossentropy

In [ ]:
def get_batch(full_x, full_y,tokenizer, batch_size, num):
    while 1:
        cnt = 0
        ind = []
        seg = []
        mas = []
        Y = []
        for i in range(num):
            try:
                indices,segments = tokenizer.encode(first=full_x[i],max_len=MAX_LEN)
            #masked = [1] * len(full_x[i]) + [0] * (max_len-len(full_x[i]))
                ind.append(indices)
                seg.append(segments)
                Y.append(full_y[i])
                cnt += 1
            except Exception:
                pass
            if cnt == batch_size:
                cnt = 0
                Y = to_categorical(Y, 2) ## one hot 编码
                yield ([np.array(ind),np.array(seg)], np.array(Y))
                ind = []
                seg = []
                mas = []
                Y = []

In [ ]:
def preprocess(data):
    data.dropna(inplace=True)
    data = data[data.text != '']
    
    pos = data.loc[data['label'] == 1.0]
    neg = data.loc[data['label'] == 0.0]
    
    num_data = min(len(pos), len(neg)) #number of samples for each label
    
    pos = pos[0: num_data]
    neg = neg[0: num_data]

    resampled = pos.append(neg)
    
    return resampled

In [ ]:
nusconv = pd.read_csv('nusconv_nopunc.csv')
chatbot = pd.read_csv('chatbot.csv')

nus_conv = preprocess(nusconv)
chat_bot = preprocess(chatbot)

shuffled = nus_conv.append(chat_bot)
shuffled = shuffled.sample(len(shuffled), random_state=1)
shuffled.reset_index(inplace=True)
shuffled = shuffled.loc[:, ['text', 'label']]

print(shuffled)

train_x = shuffled.loc[0:12000, 'text'].to_list()
train_y = shuffled.loc[0:12000, 'label'].to_list()

val_x = shuffled.loc[12000:, 'text'].to_list()
val_y = shuffled.loc[12000:, 'label'].to_list()



print(len([y for y in val_y if y == 0]))

In [ ]:
from keras.models import load_model
from keras_bert import get_custom_objects
#del model
model = load_model('bert/bert_model_0525.h5', custom_objects=get_custom_objects())

In [ ]:
model.fit_generator(
    generator=get_batch(train_x, train_y,tokenizer, BATCH_SIZE, len(train_x)),
    steps_per_epoch=1000,
    epochs=1,
    validation_data=get_batch(val_x, val_y,tokenizer, BATCH_SIZE, len(val_x)),
    validation_steps=100,
    callbacks=[
        #keras.callbacks.EarlyStopping(monitor='val_loss', patience=3),
       keras.callbacks.ModelCheckpoint('bert/model_.h5', monitor='val_acc', period=1, verbose=1, save_best_only=True, mode='max')
    ],
)
#keras.callbacks.ModelCheckpoint('model_.h5', monitor='val_acc', verbose=1, save_best_only=True, mode='max')


In [ ]:
#model.load_weights(checkpoint_filepath)

In [ ]:
import regex as re
test = pd.read_csv('rawdata/ywj.csv')
test = test.iloc[:, 0:2]
tt = pd.read_csv('rawdata/csj.csv')
tt = tt.iloc[:, 0:2]
test['label'] = 0
sentence = []
count = 0
for example in test['疑问句'].to_list():
    if example[-1] == '？' or example[-1] == '?':
        test.loc[count] = [count, re.sub(r'\s+|[，,.。?？]\s*', '', example[:-1]), 1]
    else:
        test.loc[count] = [count, re.sub(r'\s+|[，,.。?？]\s*', '', example), 1]
    count += 1
for example in tt['陈述句'].to_list():
    if example[-1] == '？' or example[-1] == '?':
        test.loc[count] = [count, re.sub(r'\s+|[，,.。?？]\s*', '', example[:-1]), 0]
    else:
        test.loc[count] = [count, re.sub(r'\s+|[，,.。?？]\s*', '', example), 0]
    count += 1
print(test)


In [ ]:
def test_tokenize(test_data_x):
    ind = []
    seg = []
    mas = []
    for i in range(len(test_data_x)):
        try:
            indices,segments = tokenizer.encode(first=test_data_x[i],max_len=MAX_LEN)
            ind.append(indices)
            seg.append(segments)
        except Exception:
            pass

    return [np.array(ind),np.array(seg)]

x_test = test_tokenize(test['疑问句'].to_list())

In [ ]:
from memory_profiler import profile
%reload_ext memory_profiler

@profile
def memory_test():
    return model.predict(x_test)

memory_test()

In [ ]:
from memory_profiler import LogFile
import sys
sys.stdout = LogFile('memory_profile_log')
print(sys.stdout)

In [ ]:
rpred = model.predict(x_test)
y_test = test['label'].to_list()
rpred = pd.DataFrame(rpred)
rpred = rpred.iloc[:, 0]

pred = rpred.copy()

print(rpred)
pred[pred >= 0.5] = 0
pred[pred != 0] = 1
print(pred)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve
print("Accuracy: ", accuracy_score(y_test, pred.to_list()))
print("Precision: ", precision_score(y_test, pred))
print("Recall: ", recall_score(y_test, pred))

In [ ]:
model.save('bert/bert_model_0527.h5')

# Plotting

## Model

In [ ]:
print(model.summary)

In [ ]:
from keras.utils.vis_utils import plot_model
import pydot

plot_model(model, to_file='plot_model.png', show_shapes=True, show_layer_names=True)

## Training History

In [ ]:
from sklearn.metrics import plot_roc_curve, confusion_matrix, plot_confusion_matrix
import seaborn as sn
import matplotlib.pyplot as plt

plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])

plt.title = 'Model Loss'
plt.ylabel = 'Loss'
plt.xlabel = 'Epoch'

plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(model.history.history['accuracy'])
plt.plot(model.history.history['val_accuracy'])

plt.title = 'Model Accuracy'
plt.ylabel = 'Accuracy'
plt.xlabel = 'Epoch'

plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(model.history.history['auc_2'])
plt.plot(model.history.history['val_auc_2'])

plt.title = 'Model AUC'
plt.ylabel = 'AUC'
plt.xlabel = 'Epoch'

plt.legend(['train', 'test'], loc='upper left')
plt.show()

## Confusion Matrix

In [ ]:
cm = confusion_matrix(y_test, pred, labels=[0, 1])
print(cm)

df_confusion = pd.DataFrame(cm, index = [0, 1], columns = [0, 1])
plt.figure(figsize = (10,7))
sn.heatmap(df_confusion, annot=True, cmap="Blues", fmt="g")